In [1]:
import pandas as pd

In [2]:
# Generate sample datetime data
date_rng = pd.date_range(start='2022-01-01', end='2022-12-31', freq='H')
data = {'datetime': date_rng}
df = pd.DataFrame(data)
df['value'] = range(len(date_rng))

In [3]:
# Example 1: Convert datetime to a different time zone
df['datetime_utc'] = df['datetime'].dt.tz_localize('UTC')
df['datetime_new_york'] = df['datetime_utc'].dt.tz_convert('America/New_York')


In [4]:
# Example 2: Handling missing time zone information
df_missing_tz = df[df.index % 3 == 0]
df_missing_tz['datetime_missing_tz'] = df_missing_tz['datetime'].dt.tz_localize(None)


C:\Users\rvign\AppData\Local\Temp\ipykernel_21620\3968536486.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_missing_tz['datetime_missing_tz'] = df_missing_tz['datetime'].dt.tz_localize(None)


In [5]:
# Example 3: Handling daylight saving time transitions
spring_dst = pd.Timestamp('2022-03-12 01:59:59', tz='US/Eastern')
fall_dst = pd.Timestamp('2022-11-05 01:59:59', tz='US/Eastern')
df['is_dst_spring'] = df['datetime_new_york'] == spring_dst
df['is_dst_fall'] = df['datetime_new_york'] == fall_dst


In [6]:
# Example 4: Convert time zones for entire DataFrame
df['datetime_london'] = df['datetime_utc'].dt.tz_convert('Europe/London')
df['datetime_tokyo'] = df['datetime_utc'].dt.tz_convert('Asia/Tokyo')


In [7]:
# Example 5: Handling daylight saving time transitions using DST rules
eastern = df['datetime_new_york'].dt.tz_localize(None)
spring_dst_rules = eastern.dt.tz_localize('US/Eastern', ambiguous='NaT', nonexistent='NaT')
fall_dst_rules = eastern.dt.tz_localize('US/Eastern', ambiguous='infer', nonexistent='shift_forward')
df['spring_dst_rules'] = spring_dst_rules
df['fall_dst_rules'] = fall_dst_rules


In [8]:
# Example 6: Handling time zones when reading from CSV
df.to_csv('output-contends/timezone_example.csv', index=False)
df_from_csv = pd.read_csv('output-contends/timezone_example.csv', parse_dates=['datetime', 'datetime_utc', 'datetime_new_york'])


In [9]:
# Example 7: Time zone conversion with resampling
df_resampled = df.set_index('datetime_new_york').resample('D')['value'].sum()


In [10]:
# Example 8: Time zone conversion with groupby
df_grouped = df.groupby(df['datetime_new_york'].dt.date)['value'].sum()


In [11]:
# Example 9: Working with time zones in multi-level index
df_multiindex = df.set_index(['datetime_new_york', 'is_dst_spring'])


In [12]:
# Example 10: Time zone conversion with apply
df['hour'] = df['datetime_new_york'].dt.hour
df['hour_utc'] = df['hour'].apply(lambda x: pd.Timestamp(x, tz='UTC'))


In [13]:
# Example 11: Using pytz library for custom time zones
import pytz
df['custom_tz'] = df['datetime'].dt.tz_localize(pytz.timezone('Europe/Berlin'))


NonExistentTimeError: 2022-03-27 02:00:00

In [14]:
# Example 12: Time zone arithmetic
df['time_diff'] = df['datetime_new_york'] - df['datetime_new_york'].shift(1)


In [15]:
# Example 13: Converting time zone strings to time zones
df['timezone'] = df['datetime_new_york'].dt.tz.zone


In [16]:
# Example 14: Handling time zone conversion ambiguity
ambiguous_time = pd.Timestamp('2022-11-06 01:30:00', tz='US/Eastern')
ambiguous_time_dst = ambiguous_time.tz_localize(None)
ambiguous_time_eastern = ambiguous_time_dst.tz_localize('US/Eastern', ambiguous='NaT')


AmbiguousTimeError: Cannot infer dst time from 2022-11-06 01:30:00, try using the 'ambiguous' argument

In [17]:
# Example 15: Handling missing time zone conversion
missing_tz_time = pd.Timestamp('2022-07-01 12:00:00')
missing_tz_time_new_york = missing_tz_time.tz_localize('US/Eastern', nonexistent='shift_forward')
